In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install litellm


In [ ]:
#Pongo mi token

from huggingface_hub import login
login("") #<- Por seguridad no me deja hacer push origin sin eliminar mi ID


In [ ]:
# =====================================================
# Cargar agentes ya creados
# =====================================================
from src.agents import get_agents
researcher, writer, reviewer = get_agents()

# =====================================================
# Herramienta de búsqueda
# =====================================================
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

query = "Impact of synthetic data in healthcare site:medium.com OR site:researchgate.net"
results = search_tool.run(query)

print(results[:500])

# =====================================================
# Writer usando chat_completion
# =====================================================
from huggingface_hub import InferenceClient
client = InferenceClient(model="HuggingFaceH4/zephyr-7b-beta")

writer_prompt = f"""
You are an AI research writer. Using the following web snippets:

{results}

Write a 500-word Markdown report on "Impact of Synthetic Data in Healthcare".

Structure:
1. Introduction
2. Key Findings
3. Ethical & Technical Challenges
4. Conclusion

Ensure coherence, clarity, and academic tone.
"""

writer_resp = client.chat_completion(
    model="HuggingFaceH4/zephyr-7b-beta",
    messages=[{"role": "user", "content": writer_prompt}],
    max_tokens=700,
    temperature=0.3
)

writer_output = writer_resp["choices"][0]["message"]["content"]
print("\n---- WRITER OUTPUT ----\n")
print(writer_output[:800])


# =====================================================
# Reviewer usando chat_completion
# =====================================================
review_prompt = f"""
Review the following text for academic quality, coherence, and factual quality.
Provide a structured critique.

Text:
{writer_output}
"""

review_resp = client.chat_completion(
    model="HuggingFaceH4/zephyr-7b-beta",
    messages=[{"role": "user", "content": review_prompt}],
    max_tokens=300,
    temperature=0.2
)

review_output = review_resp["choices"][0]["message"]["content"]

print("\n---- REVIEW ----\n")
print(review_output)


# =====================================================
# Guardar archivo
# =====================================================
with open("research_summary.md", "w", encoding="utf-8") as f:
    f.write(writer_output)

print("\n📄 Archivo 'research_summary.md' guardado exitosamente.")
